In [3]:
import pandas as pd
import numpy as np
import yfinance as yf

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [4]:
msft = yf.Ticker("AAPL")
print(msft)

yfinance.Ticker object <AAPL>


In [5]:
msft.history(period="5d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-03-18,122.879997,123.180000,120.320000,120.529999,121229700,0,0
2021-03-19,119.900002,121.430000,119.680000,119.989998,185023200,0,0
2021-03-22,120.330002,123.870003,120.260002,123.389999,111912300,0,0
2021-03-23,123.330002,124.239998,122.139999,122.540001,95195700,0,0
2021-03-24,122.820000,122.900002,120.065002,120.089996,85447491,0,0


In [6]:
series = msft.history(period = "100d")
# series = pd.Series([1,2,3,4,5,6,7,8,9,10])

In [7]:
series.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-10-29,112.009093,116.554445,111.839634,114.949615,146129200,0.0,0
2020-10-30,110.703292,111.630305,107.374023,108.510361,190272600,0.0,0
2020-11-02,108.759566,110.324523,106.975314,108.420654,122866900,0.0,0
2020-11-03,109.307795,111.131912,108.380782,110.085289,107624400,0.0,0
2020-11-04,113.773407,115.218747,111.989155,114.580803,138235500,0.0,0


In [8]:
aapl = yf.download(tickers = ['AAPL'],
                  period='100d')

[*********************100%***********************]  1 of 1 completed


#### Weird, data from two different methods in yfinance are returning slightly different open and close values. The adjusted close value matches the close value. 

### Define functions for technical markers

In [9]:
# note trail_size includes the current record. 
def compute_highest_high(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the hioghest high. """
    return series['High'].rolling(window=trail_size, min_periods=1).max()

In [10]:
def compute_lowest_low(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the lowest low. """
    return series['Low'].rolling(window=trail_size, min_periods=1).min()

In [11]:
def compute_avg_vol(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the average volume. """
    return series['Volume'].rolling(window=trail_size, min_periods=1).mean()

In [12]:
def compute_sma(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the simple moving average. """
    return series['Close'].rolling(window=trail_size, min_periods=1).mean()

In [13]:
def compute_sd(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the standard deviation. """
    return series['Close'].rolling(window=trail_size, min_periods=1).std()

In [14]:
def compute_willr(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Williams %R marker. """
    return -100*((series['High']-series['Close'])/(series['High']-series['Low']))

In [15]:
def compute_atr(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Average True Range. """
    
    # get a series of the previous day's closes
    prev_closes = series['Close'].shift(1).fillna(method='bfill')
    
    # make a df of the 3 differences we want to max
    triple_diffs = pd.DataFrame()
    triple_diffs['highlow'] = series['High'] - series['Low']
    triple_diffs['highclose'] = abs(series['High'] - prev_closes)
    triple_diffs['closelow'] = abs(prev_closes - series['Low'])
    
    true_ranges = triple_diffs.max(axis=1)
    
    return true_ranges.rolling(window=trail_size, min_periods=1).mean()

In [16]:
def compute_dmh(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Directional Movement High. """
    return series['High'].diff(1).fillna(0)

In [17]:
def compute_dml(series):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series Directional Movement Low. """
    return series['Low'].diff(1).fillna(0)

In [18]:
def compute_ema(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the exponential moving average. """
    
    return series['Close'].ewm(span=trail_size).mean()

In [19]:
def compute_wma(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the weighted moving average. """
    
    weights = np.arange(1, trail_size+1)
    weights = weights[::-1]
    #should produce array [trailsize+1, trailsize, ..., 2, 1]
    
    #SUM ((close[t]*N) + (close[t-1] *(N-1)) + ... + (close[t-N])) 
    #wma = wma/(sum of the weights)
    wma = series['Close'].rolling(window=trail_size).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
    
    return wma

In [20]:
def compute_bbhigh(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the High Bollinger Bands. """
    #SMA[N] + SD[N]*2

    return series['SMA'] + series['SD']*2

In [21]:
def compute_bblow(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Low Bollinger Bands. """
    #SMA[N] - SD[N]*2
    
    return series['SMA'] - series['SD']*2

In [22]:
def compute_perbhigh(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Higher Limit Bollinger Bands. """
    #BBHIGH[N]/SD[N]
    return series['BBHIGH'].div(series['SD'])

In [23]:
def compute_perblow(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the Lower Limit Bollinger Bands. """
    #BBLOW[N]/SD[N]
    return series['BBLOW'].div(series['SD'])

In [24]:
def compute_trima(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the triangular moving average. """
    #(SMA[t-N] + ... + SMA[t])/N
    return (series['SMA'].rolling(window=trail_size).sum() / trail_size)

In [25]:
def compute_rsi(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the relative strength index. 
    This code was taken off stackoverflow. It produces warnings but
    does what we need it to do. Can't seem to figure out why it returns
    series rsi and RSI in the table"""
    
    change = pd.DataFrame()
    change = series['Close'].diff(1) # Calculate change

    # calculate gain / loss from every change
    series['Gain'] = np.select([change>0, change.isna()], 
                        [change, np.nan], 
                        default=0) 
    series['Loss'] = np.select([change<0, change.isna()], 
                        [-change, np.nan], 
                        default=0)

    # create avg_gain /  avg_loss columns with all nan
    series['Avg_gain'] = np.nan 
    series['Avg_loss'] = np.nan


    # keep first occurrence of rolling mean
    series['Avg_gain'][trail_size] = series['Gain'].rolling(window=trail_size).mean().dropna().iloc[0] 
    series['Avg_loss'][trail_size] = series['Loss'].rolling(window=trail_size).mean().dropna().iloc[0]
    
    #Looping through the pandas series
    for i in range(trail_size+1, series.shape[0]):
        series['Avg_gain'].iloc[i] = (series['Avg_gain'].iloc[i-1] * (trail_size - 1) + series['Gain'].iloc[i]) / trail_size
        series['Avg_loss'].iloc[i] = (series['Avg_loss'].iloc[i-1] * (trail_size - 1) + series['Loss'].iloc[i]) / trail_size

    # calculate rs and rsi
    series['rs'] = series['Avg_gain'] / series['Avg_loss']
    rsi = 100 - (100 / (1 + series['rs'] ))

    del series['Gain']
    del series['Loss']
    del series['Avg_gain']
    del series['Avg_loss']
    del series['rs']

    return rsi

In [26]:
def compute_dx(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the directional index. """
    #(abs(+DM - -DM)/(+DM + -DM))*100 
    
    num = abs(series['DMH'] - series['DML'])
    den = abs(series['DMH'] + series['DML'])

    return num / den * 100

In [27]:
def compute_positive_di(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the positive directional indicator. """
    #(+DM/ATR[N])*100 
    
    return (series['DMH'].div(series['ATR']).mul(100))

In [28]:
def compute_negative_di(series, trail_size=20):
    """A function to take in a dataframe for one stock time series, 
    and return a pandas series for the negative directional indicator. """
    #(-DM/ATR[N])*100 
    
    return (series['DML'].div(series['ATR']).mul(100))

In [29]:
def compute_adx(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Average Directional Index."""
    
    # iterative solution given recursive formula
    # init to zero. 
    adx_vals = np.zeros(len(series))
    for i in range(1, len(adx_vals)):
        adx_vals[i] = (adx_vals[i-1]*(trail_size-1)+series['DX'][i]) / trail_size
        
    return adx_vals

In [30]:
def compute_roc(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Rate of Change."""
    return (series['Close']/(series['Close'].shift(trail_size).fillna(method='bfill'))-1)*100

In [31]:
def compute_macd(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Moving Average Convergance Divergence."""
    
    return compute_ema(series, trail_size) - compute_ema(series, 2*trail_size)

In [32]:
def compute_cci(series,trail_size = 20):
    """A function to take in a dataframe for one stock time series,
    and return a panda series Commonodity Channel Index."""
    
    num = (series['High'] + series['Low'] + series['SMA'])/3
    
    return (num - series['SMA']) / (0.015*series['SD'])

In [33]:
#marking down here, I've added the following Technical markers

In [48]:
def compute_adl(series, trail_size = 20):
    """A function to take the dataframe of one stock series
    and return a panda series """
    
    #define and obtain CMFV value, note that function is recursive
    #note, to adjust period of ADL, adjust trail_size for each parameter
    
    #closing price on that instance of the day
    Pc = series['Close']
    Pc_prev = series['Close'].shift(1).fillna(method='bfill')
    
    #low price for that period
    Pl = series['Low']
    Pl_prev = series['Low'].shift(1).fillna(method='bfill')
    
    #high price for that period
    Ph = series['High']
    Ph_prev = series['High'].shift(1).fillna(method='bfill')
    
    #volume for that period
    V = series['Volume']
    V_prev = series['Volume'].shift(1).fillna(method='bfill')
    
    CMFV = (((Pc - Pl) - (Ph - Pc))/(Ph - Pl)) * V
    CMFV_prev = (((Pc_prev - Pl_prev) - (Ph_prev - Pc_prev))/(Ph_prev - Pl_prev)) * V_prev
    
    #previous series
    #prev_closes = series['Close'].shift(1).fillna(method='bfill')
    return CMFV + CMFV_prev 
    #working, and here is why. The ADL shows buying/selling pressure over a timeframe. Strong negative values indicate large selling pressure while strong positive values indicate large buying pressure
    #in this case, the returned values are strong positive/negative numbers that correlate to strong selling or buying forces. The data is valid

#print(compute_adl(series, trail_size=1)*8)
#print("BREAK")
#print(compute_adl(series, trail_size=8))
#Testing method above returns different values indicating recursion has an effect this time around

In [49]:
def compute_obv(series):
    return np.where(series['Close'] > series['Close'].shift(1), series['Volume'], np.where(series['Close'] < series['Close'].shift(1), -series['Volume'], 0)).cumsum()

#should be working, taken from stack overflow

In [54]:
def compute_vpt(series, trail_size = 20):
    #x is the close[t - 1], borrowing from Jacob, that is prev_closes = series['Close'].shift(1).fillna(method='bfill')
    x = series['Close'].shift(1).fillna(method='bfill')
    inter = series['Volume'] * (series['Close'] - x)/x
    
    y = series['Close'].shift(2).fillna(method='bfill')
    inter_prev = series['Volume'].shift(1).fillna(method = 'bfill') * (series['Close'].shift(1).fillna(method = 'bfill') - y)/y
    
    return inter_prev + inter

#print(compute_vpt(series))

"""
print(compute_vpt(series, trail_size=1)*8)
print("BREAK")
print(compute_vpt(series, trail_size=8))
"""    
    
    #possibly working?

'\nprint(compute_vpt(series, trail_size=1)*8)\nprint("BREAK")\nprint(compute_vpt(series, trail_size=8))\n'

In [39]:
def compute_mfi(series, trail_size = 20):
    #works
    return series['Volume'] * ((series['High'] + series['Low'] + series['Close'])/3)
#returns the actual money flowed, not the actual index. to get the index you do some quick math to express it out of 100

#### A function to add markers to a given series

In [155]:
def add_technical_markers(series, trail_size=20):
    series['HH'] = compute_highest_high(series, trail_size)
    series['LL'] = compute_lowest_low(series, trail_size)
    series['AV'] = compute_avg_vol(series, trail_size)
    series['SMA'] = compute_sma(series, trail_size)
    series['SD'] = compute_sd(series, trail_size)
    series['WILLR'] = compute_willr(series)
    series['ATR'] = compute_atr(series, trail_size)
    series['DMH'] = compute_dmh(series)
    series['DML'] = compute_dml(series)
    series['EMA'] = compute_ema(series, trail_size)
    series['WMA'] = compute_wma(series, trail_size)
    series['BBHIGH'] = compute_bbhigh(series, trail_size)
    series['BBLOW'] = compute_bblow(series, trail_size)
    series['PERBHIGH'] = compute_perbhigh(series, trail_size)
    series['PERBLOW'] = compute_perblow(series, trail_size)
    series['TRIMA'] = compute_trima(series, trail_size)
    series['RSI'] = compute_rsi(series, trail_size)
    series['DX'] = compute_dx(series, trail_size)
    series['PDI'] = compute_positive_di(series, trail_size)
    series['NDI'] = compute_negative_di(series, trail_size)
#     series['ADX'] = compute_adx(series,trail_size = 20)
    series['ROC'] = compute_roc(series,trail_size = 20)
    series['MACD'] = compute_macd(series,trail_size = 20)
    series['CCI'] = compute_cci(series,trail_size = 20)
    series['ADL'] = compute_adl(series, trail_size = 20)
    series['OBV'] = compute_obv(series)
    series['VBT'] = compute_vpt(series, trail_size = 20)
    series['MFI'] = compute_mfi(series, trail_size = 20)
    return series

In [156]:
series = add_technical_markers(series, trail_size=3)

<ipython-input-139-32fbadae6c78>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['Avg_gain'][trail_size] = series['Gain'].rolling(window=trail_size).mean().dropna().iloc[0]
<ipython-input-139-32fbadae6c78>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series['Avg_loss'][trail_size] = series['Loss'].rolling(window=trail_size).mean().dropna().iloc[0]
C:\Users\polob\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [157]:
series.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ROC,MACD,CCI,ADL,OBV,VBT,MFI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-15,118.338697,120.810728,117.770528,120.322304,112559200,0.0,0,120.810728,117.770528,1.125592e+08,120.322304,NaN,-16.065528,3.040199,0.000000,0.000000,120.322304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,1.527855e+09,0,0.000000e+00,1.346597e+10
2020-10-16,120.890469,121.159606,118.428401,118.637726,115393800,0.0,0,121.159606,117.770528,1.139765e+08,119.480015,1.191176,-92.335815,2.885702,0.348878,0.657873,119.199252,NaN,121.862368,117.097662,102.304209,98.304209,NaN,NaN,30.692227,12.089898,22.797668,-1.400055,-0.021057,11.715363,-1.954116e+09,-115393800,-3.231152e+07,1.377901e+10
2020-10-19,119.574711,120.033233,115.288526,115.607498,120639300,0.0,0,121.159606,115.288526,1.161974e+08,118.189176,2.389194,-93.277310,3.505370,-1.126374,-3.139875,117.146821,118.974977,122.967563,113.410789,51.468230,47.468230,119.330498,NaN,47.196059,-32.132803,-89.573275,-3.918480,-0.079026,-9.827519,-2.088378e+09,-236033100,-6.162703e+07,1.411195e+10
2020-10-20,115.826793,118.597871,115.258624,117.132591,124423700,0.0,0,121.159606,115.258624,1.201523e+08,117.125938,1.515125,-43.880554,3.605053,-1.435361,-0.029902,117.139232,117.376794,120.156188,114.095689,79.304483,75.304483,118.265043,24.440991,95.918575,-39.815267,-0.829442,-2.650974,-0.075407,-5.799026,3.045617e+08,-111609400,3.282793e+07,1.455712e+10
2020-10-21,116.295282,118.328731,116.075987,116.494644,89946000,0.0,0,120.033233,115.258624,1.116697e+08,116.411578,0.765932,-81.415690,3.445566,-0.269140,0.817363,116.806541,116.263720,117.943442,114.879713,153.986779,149.986779,117.242231,21.191210,198.186529,-7.811209,23.722169,-3.181172,-0.086121,45.886346,-1.130286e+09,-201555400,-9.797579e+06,1.052066e+10


In [158]:
top100tech = ['AAPL',
'MSFT',
'TSM',
'NVDA',
'INTC',
'ASML',
'ADBE',
'CRM',
'AVGO',
'ORCL',
'CSCO',
'ACN',
'TXN',
'SHOP',
'QCOM',
'SAP',
'SNE',
'AMAT',
'INTU',
'NOW',
'SQ',
'IBM',
'MU',
'AMD',
'UBER',
'FIS',
'LRCX',
'FISV',
'SNOW',
'INFY',
'TEAM',
'ADSK',
'DELL',
'WDAY',
'VMW',
'ADI',
'NXPI',
'KLAC',
'CRWD',
'PLTR',
'DOCU',
'TEL',
'ERIC',
'MCHP',
'CDNS',
'CTSH',
'APH',
'SNPS',
'HPQ',
'PANW',
'OKTA',
'RNG',
'STM',
'MRVL',
'XLNX',
'WIT',
'MSI',
'SWKS',
'GLW',
'DDOG',
'ANSS',
'U',
'ZS',
'FTNT',
'STNE',
'ZBRA',
'KEYS',
'FTV',
'MXIM',
'COUP',
'GRMN',
'SPLK',
'HUBS',
'WORK',
'NET',
'FLT',
'CDW',
'CAJ',
'AFRM',
'PAYC',
'TER',
'VRSN',
'WDC',
'ZI',
'UMC',
'ANET',
'EPAM',
'UI',
'XM',
'GIB',
'TRMB',
'TYL',
'WIX',
'HPE',
'LYFT',
'LOGI',
'MPWR']

In [159]:
# tickers = pd.Series(top100tech).apply(yf.Ticker)

In [160]:
# hacky way to get lots of data
# hist_datas = tickers.apply(lambda x: x.history(period="5d"))

In [161]:
' '.join(top100tech)

'AAPL MSFT TSM NVDA INTC ASML ADBE CRM AVGO ORCL CSCO ACN TXN SHOP QCOM SAP SNE AMAT INTU NOW SQ IBM MU AMD UBER FIS LRCX FISV SNOW INFY TEAM ADSK DELL WDAY VMW ADI NXPI KLAC CRWD PLTR DOCU TEL ERIC MCHP CDNS CTSH APH SNPS HPQ PANW OKTA RNG STM MRVL XLNX WIT MSI SWKS GLW DDOG ANSS U ZS FTNT STNE ZBRA KEYS FTV MXIM COUP GRMN SPLK HUBS WORK NET FLT CDW CAJ AFRM PAYC TER VRSN WDC ZI UMC ANET EPAM UI XM GIB TRMB TYL WIX HPE LYFT LOGI MPWR'

In [162]:
data = yf.download(tickers = ' '.join(top100tech),
                  period='365d')

[*********************100%***********************]  97 of 97 completed


In [163]:
data.head(8)

Adj Close                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             \
                 AAPL         ACN        ADBE         ADI        ADSK AFRM       AMAT        AMD        ANET        ANSS        APH        ASML        AVGO        CAJ       CDNS         CDW        COUP         CRM       CRWD       CSCO       CTSH       DDOG       DELL       DOCU        EPAM      ERIC         FIS        FISV         FLT       FTNT        FTV        GIB        GLW       GRMN        HPE        HPQ        HUBS         IBM       INFY       INTC        INTU       KEYS        KLAC       LOGI        LRCX       LYFT       MCHP        MPWR       MRVL        MSFT         MSI         MU       MXIM        NET         NOW        NVDA        NXPI        OKTA       ORCL        PANW        PAYC PLTR       QCOM         RNG         SAP        SHOP        SNE SNOW        SNPS        SPLK         SQ        STM       STNE       SWKS        TEAM        TEL        TER       TRMB        TSM         TXN         TYL   U       UBER          UI       UMC         VMW        VRSN        WDAY   
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2019-09-27  53.993587  186.911758  277.540009  107.821159  145.190002  NaN  48.511913  28.719999  242.779999  217.089996  47.259415  240.500687  259.536041  26.990000  65.199997  119.603264  127.889999  148.259995  53.459999  46.485378  57.598850  32.270000  50.950001  60.639999  180.360001  7.831548  129.064453  102.610001  288.000000  76.220001  57.060062  78.419998  27.153677  82.167747  14.248562  17.712589  149.910004  132.739746  10.803939  48.979469  260.256561  96.290001  154.495819  39.918694  224.247223  41.349998  89.449715  154.938339  24.466721  135.511932  166.645081  43.209999  55.420467  18.240000  250.259995  171.350616  104.820015   97.059998  52.717087  202.399994  206.399994  NaN  73.790382  123.849998  115.768707  305.690002  58.770000  NaN  136.210007  118.059998  60.750000  18.876280  34.299999  75.787910  123.029999  90.034462  56.982651  38.220001  44.023319  121.842903  256.260010 NaN  30.290001  114.853279  1.988122  145.880005  187.250000  168.649994   
2019-09-30  55.264339  187.8

In [164]:
df = pd.melt(data, ignore_index=False)
df.columns = ['Variable', 'Ticker', 'Value']

In [165]:
# this is a nicely formed dataframe of historical data
nice_df = df.reset_index().pivot(index=['Ticker', 'Date'], columns=['Variable'], values=['Value'])

In [166]:
nice_df.columns = nice_df.columns.droplevel()

In [167]:
nice_df.head(5)

Variable           Adj Close      Close       High        Low       Open       Volume
Ticker Date                                                                          
AAPL   2019-09-27  53.993587  54.705002  55.240002  54.320000  55.134998  101408000.0
       2019-09-30  55.264339  55.992500  56.145000  55.197498  55.224998  103909600.0
       2019-10-01  55.417320  56.147499  57.055000  56.049999  56.267502  139223200.0
       2019-10-02  54.028126  54.740002  55.895000  54.482498  55.764999  138449200.0
       2019-10-03  54.487080  55.205002  55.240002  53.782501  54.607498  114426000.0

In [168]:
df = nice_df.reset_index()

In [169]:
df.columns.name=None

In [170]:
df.head(10)

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume
0,AAPL,2019-09-27,53.993587,54.705002,55.240002,54.320000,55.134998,101408000.0
1,AAPL,2019-09-30,55.264339,55.992500,56.145000,55.197498,55.224998,103909600.0
2,AAPL,2019-10-01,55.417320,56.147499,57.055000,56.049999,56.267502,139223200.0
3,AAPL,2019-10-02,54.028126,54.740002,55.895000,54.482498,55.764999,138449200.0
4,AAPL,2019-10-03,54.487080,55.205002,55.240002,53.782501,54.607498,114426000.0
5,AAPL,2019-10-04,56.014454,56.752499,56.872501,55.972500,56.410000,138478800.0
6,AAPL,2019-10-07,56.026794,56.764999,57.482498,56.459999,56.567501,122306000.0
7,AAPL,2019-10-08,55.370438,56.099998,57.014999,56.082500,56.455002,111820000.0
8,AAPL,2019-10-09,56.019390,56.757500,56.947498,56.410000,56.757500,74770400.0
9,AAPL,2019-10-10,56.774445,57.522499,57.610001,56.825001,56.982498,113013600.0


In [171]:
# df[df['Ticker']=='AAPL']

In [172]:
# df[df['Close']>126]

In [173]:
df.groupby('Ticker').mean().head()

,Adj Close,Close,High,Low,Open,Volume
Ticker,,,,,,
AAPL,93.663312,94.114650,95.324144,92.782048,94.073699,1.419044e+08
ACN,212.836477,214.922301,217.149452,212.626233,214.806082,2.176883e+06
ADBE,400.880165,400.880165,405.982192,394.976439,400.988132,2.928957e+06
ADI,119.341250,121.119452,122.716986,119.478712,121.187370,2.808694e+06
ADSK,219.339452,219.339452,222.729918,215.774630,219.262055,1.600302e+06


In [174]:
# compute all our technical markers for each of the stocks. 
dataset = df.groupby('Ticker').apply(add_technical_markers)

In [175]:
dataset.head()

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ROC,MACD,CCI,ADL,OBV,VBT,MFI
0,AAPL,2019-09-27,53.993587,54.705002,55.240002,54.320000,55.134998,101408000.0,55.240002,54.320000,1.014080e+08,54.705002,NaN,-58.152032,0.920002,0.000000,0.000000,54.705002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,-3.306725e+08,0.0,0.000000e+00,5.552595e+09
1,AAPL,2019-09-30,55.264339,55.992500,56.145000,55.197498,55.224998,103909600.0,56.145000,54.320000,1.026588e+08,55.348751,0.910399,-16.094967,1.180000,0.904999,0.877499,55.380939,NaN,57.169549,53.527953,62.796153,58.796153,NaN,NaN,1.542788,76.694792,74.364271,2.353530,0.016094,15.743932,1.409223e+09,103909600.0,4.891087e+07,5.795904e+09
2,AAPL,2019-10-01,55.417320,56.147499,57.055000,56.049999,56.267502,139223200.0,57.055000,54.320000,1.148469e+08,55.615000,0.791883,-90.298533,1.140834,0.910000,0.852501,55.662415,NaN,57.198767,54.031234,72.231306,68.231306,NaN,NaN,3.262349,79.766225,74.726144,2.636865,0.023535,52.617143,-2.244196e+09,243132800.0,7.707970e+06,7.854625e+09
3,AAPL,2019-10-02,54.028126,54.740002,55.895000,54.482498,55.764999,138449200.0,57.055000,54.320000,1.207475e+08,55.396251,0.780678,-81.769693,1.271875,-1.160000,-1.567501,55.396129,NaN,56.957607,53.834894,72.959133,68.959133,NaN,NaN,14.940461,-91.203892,-123.243290,0.063979,-0.000904,-11.813170,-1.759395e+09,104683600.0,-6.941249e+07,7.620129e+09
4,AAPL,2019-10-03,54.487080,55.205002,55.240002,53.782501,54.607498,114426000.0,57.055000,53.782501,1.194832e+08,55.358001,0.681476,-2.401361,1.309000,-0.654999,-0.699997,55.349897,NaN,56.720952,53.995050,83.232541,79.232541,NaN,NaN,3.320909,-50.038089,-53.475686,0.913993,-0.004796,-55.223263,2.178609e+09,219109600.0,1.944030e+07,6.263965e+09


In [176]:
dataset.tail()

,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ROC,MACD,CCI,ADL,OBV,VBT,MFI
35400,ZS,2021-03-04,180.899994,180.899994,190.960007,175.679993,184.330002,3171400.0,230.880997,175.679993,2032355.00,212.340999,12.945482,-65.837720,11.147601,-11.159988,-11.780014,206.146576,217.420666,238.231963,186.450034,18.402711,14.402711,212.709449,NaN,2.702814,-100.111126,-105.673091,-13.676276,1.902265,-99.634927,-2.009110e+07,123919200.0,-2.340183e+06,5.788228e+08
35401,ZS,2021-03-05,175.369995,175.369995,185.539993,165.570007,185.539993,3940900.0,230.880997,165.570007,2167605.00,210.421999,15.347455,-50.926416,11.800350,-5.420013,-10.109985,203.215473,217.103713,241.116909,179.727088,15.710546,11.710546,212.892724,NaN,30.199435,-45.930954,-85.675299,-17.955558,0.379665,-100.970769,-1.460365e+06,119978300.0,-2.409417e+06,6.916017e+08
35402,ZS,2021-03-08,167.839996,167.839996,181.699997,166.886002,175.759995,2729100.0,230.880997,165.570007,2203170.00,207.974998,17.959589,-93.560179,12.135550,-3.839996,1.315994,199.846380,216.265142,243.894176,172.055821,13.580165,9.580165,212.894324,NaN,204.278382,-31.642541,10.844125,-22.575885,-1.282315,-83.352561,-4.755203e+07,117249200.0,-2.343630e+06,4.697927e+08
35403,ZS,2021-03-09,181.429993,181.429993,182.789993,175.000000,178.714996,2822300.0,230.880997,165.570007,2286150.00,206.252998,18.794443,-17.458303,12.524550,1.089996,8.113998,198.092439,215.349237,243.841884,168.664113,12.974148,8.974148,212.763574,NaN,76.314712,8.702878,64.784751,-15.954048,-2.075345,-64.695250,3.673697e+07,120071500.0,4.570430e+06,5.072802e+08
35404,ZS,2021-03-10,179.600006,179.600006,188.830002,178.809998,185.050003,1914465.0,230.880997,165.570007,2326098.25,204.232999,19.400467,-92.115687,12.776550,6.040009,3.809998,196.331255,213.847618,243.033933,165.432064,12.527221,8.527221,212.478574,NaN,22.639691,47.274176,29.820239,-18.363634,-2.833223,-46.764049,-3.225160e+07,118157035.0,-3.862035e+05,3.492239e+08
